# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [2]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [3]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    
    col = ["Title", "Year", "Synopsis", "Review"]
    
    df_eng.columns = col
    df_fr.columns = col
    df_sp.columns = col
    
    df_eng["Original Language"] = "English"
    df_fr["Original Language"] = "French"
    df_sp["Original Language"] = "Spanish"
    
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    
    return df

df = preprocess_data()

In [4]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",French
16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",French
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
27,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",Spanish
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",Spanish
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",Spanish
13,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous...",French
15,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""Je n'ai pas aimé ce film du tout. Le concept ...",French
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [5]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [6]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df[df["Original Language"] == "French"]["Review"].tolist()
fr_reviews_en = translate(fr_reviews, fr_en_model, fr_en_tokenizer)

# filter synopsis in French and translate to English
fr_synopsis = df[df["Original Language"] == "French"]["Synopsis"].tolist()
fr_synopsis_en = translate(fr_reviews, fr_en_model, fr_en_tokenizer)

# filter reviews in Spanish and translate to English
es_reviews = df[df["Original Language"] == "Spanish"]["Review"].tolist()
es_reviews_en = translate(es_reviews, es_en_model, es_en_tokenizer)

# filter synopsis in Spanish and translate to English
es_synopsis = df[df["Original Language"] == "Spanish"]["Synopsis"].tolist()
es_synopsis_en = translate(es_reviews, es_en_model, es_en_tokenizer)

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[df["Original Language"] == "French", "Review"] = fr_reviews_en
df.loc[df["Original Language"] == "French", "Synopsis"] = fr_synopsis_en
df.loc[df["Original Language"] == "Spanish", "Review"] = es_reviews_en
df.loc[df["Original Language"] == "Spanish", "Synopsis"] = es_synopsis_en

/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
20,Roma,2018,"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish
29,El Incidente,2014,"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish
24,Amores perros,2000,"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish
13,Les Choristes,2004,"""The Land is an absolutely beautiful film with...","""The Land is an absolutely beautiful film with...",French
18,Les Visiteurs en Amérique,2000,"""The Land is an absolutely beautiful film with...","""The Land is an absolutely beautiful film with...",French
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
26,Toc Toc,2017,"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish
25,Águila Roja,(2009-2016),"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [8]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "sentiment-analysis"
sentiment_classifier = pipeline(model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)
    sentiment = result[0]["label"]
    return sentiment

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [9]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df["sentiment"] = df["Review"].apply(lambda r: analyze_sentiment(r, sentiment_classifier))

In [10]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,sentiment
25,Águila Roja,(2009-2016),"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish,POSITIVE
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English,NEGATIVE
29,El Incidente,2014,"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish,POSITIVE
24,Amores perros,2000,"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish,POSITIVE
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English,NEGATIVE
21,La Casa de Papel,(2017-2021),"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish,POSITIVE
17,Astérix aux Jeux Olympiques,2008,"""The Land is an absolutely beautiful film with...","""The Land is an absolutely beautiful film with...",French,POSITIVE
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English,POSITIVE
16,La Tour Montparnasse Infernale,2001,"""The Land is an absolutely beautiful film with...","""The Land is an absolutely beautiful film with...",French,POSITIVE
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English,POSITIVE


In [11]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)